<a href="https://colab.research.google.com/github/lambdamoses/kallistobustools/blob/master/docs/tutorials/kb_visium/visium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By Lambda Moses and Lior Pachter

# Introduction
While this website is mainly about scRNA-seq, a major drawback of scRNA-seq is that the tissue needs to be dissociated into single cells prior to transcriptome profiling, thus losing spatial contexts of the cells which are often biologically relevant. Spatial transcriptomics aims to profile the transcriptome while preserving the spatial information about the cells. The field of spatial transcriptomics has seen tremendous growth since 2015. 

<figure>
<img src="https://pachterlab.github.io/LP_2021/07-current_analysis_files/figure-html/analysis-current-1.png" width="650"/>
<figcaption>Figure 7.2 from Museum of Spatial Transcriptomics book. Comparing number of publications over time in the prequel and the current eras. Bin width is 365 days.</figcaption>
</figure>

In this tutorial, the kallisto bustools framework is applied to 10X Visium, by far the most popular method for spatial transcriptomics after laser capture microdissection (LCM). 

<figure>
<img src="https://pachterlab.github.io/LP_2021/04-current_files/figure-html/n-insts-1.png" width="550"/>
<figcaption>Figure 4.7 from Museum of Spatial Transcriptomics book. Techniques used by at least 3 institutions and the number of institutions that have used them. LCM is excluded here since we have not comprehensively curated LCM liberature due to the sheer volume. See Chapter 6 of Museum of Spatial Transcriptomics for text mining of LCM related search results from PubMed and bioRxiv.</figcaption>
</figure>

In principle, kallisto bustools can be used for any sequencing based spatial transcriptomics method that stores spatial barcodes, UMIs, and reads in fastq files, including not only Visium, but also [Spatial Transcriptomics](https://science.sciencemag.org/content/353/6294/78) (ST, the forerunner of Visium) and [DBiT-seq](https://doi.org/10.1016/j.cell.2020.10.026). 

However, note that not all of the Space Ranger functionalities can be achieved with kallisto bustools, as Space Ranger has image processing functionalities to detect Visium fiducials and align the transcript spots to the H&E image, which kallisto bustools cannot do. Therefore, while we use kallisto bustools to process the Visium fastq files, at present, we need to use the spot coordinates as in the aligned H&E image from Space Ranger to get the H&E image to display properly in `Seurat` and `SpatialExperiment`. Nevertheless, kallisto bustools is valuable for analyzing Visium data, as it can be used to relatively quickly generate spliced and unspliced matrices for RNA velocity analysis and can be used to generate a transcript compatibility count (TCC) matrix which retains some isoform information.

Here we analyze a Visium dataset of a chicken heart at embryonic day 14 (D14), or Hamburger-Hamilton stage 40, when the 4 chambers have formed. Chicken is commonly used to study heart development, because of the similarities between avian and mammalian hearts, and because the chicken embryo can be cultured in vitro to facilitate observation. This dataset is from this publication:

> Mantri, M., Scuderi, G.J., Abedini-Nassab, R. et al. Spatiotemporal single-cell RNA sequencing of developing chicken hearts identifies interplay between cellular differentiation and morphogenesis. Nat Commun 12, 1771 (2021). https://doi.org/10.1038/s41467-021-21892-z

In this tutorial, we will begin with processing the fastq files and removing empty spots. Once we get the filtered gene count matrix, we will compare the kallisto bustools results with Space Ranger results, and then walk you through some concepts in spatial statistics and common types of data analyses in spatial transcriptomics, compared to their non-spatial counterparts in the standard scRNA-seq workflow which is commonly used in the spatial transcriptomics literature. We will demonstrate usage of some R packages for those spatial transcriptomics analyses and link to other relevant packages, whether written specifically for spatial transcriptomics or for spatial statistics in general. 

Intermediate proficiency in R and Tidyverse is assumed. For more information about spatial transcriptomics data analysis, see [Chapter 7 of Museum of Spatial Transcriptomics](https://pachterlab.github.io/LP_2021/current-analysis.html) and the [Analysis sheet of the Museum of Spatial Transcriptomics database](https://docs.google.com/spreadsheets/d/1sJDb9B7AtYmfKv4-m8XR7uc3XXw_k4kGSout8cqZ8bY/edit#gid=1424019374), which is more up to date than the Museum of Spatial Transcriptomics book.

# Setup
First, install packages and download references and data required to perform the analyses.

Install the kb-python wrapper of kallisto and bustools to get from fastq files to the gene count matrix. This is called from the command line and no background in Python is necessary here.

In [1]:
# Install kb (includes installing kallisto and bustools)
system("pip3 install kb-python")

Install the R packages; as many analyses are performed in this tutorial and many packages are used for all those analyses, this takes about an hour, as CRAN and Bioconductor don't provide Linux binaries and compiling C++ code, as quite commonly called from R via Rcpp, takes a while. Tidyverse is pre-installed on Google Colab. This is by far the most time consuming step in this tutorial. All the downstream analyses don't take more than about 15 minutes.

In [2]:
# Packages pre-installed on Google Colab
rownames(installed.packages())

[1] "IRdisplay"     "IRkernel"      "pbdZMQ"        "repr"         
  [5] "askpass"       "assertthat"    "backports"     "base64enc"    
  [9] "BH"            "blob"          "brew"          "brio"         
 [13] "broom"         "cachem"        "callr"         "cellranger"   
 [17] "cli"           "clipr"         "colorspace"    "commonmark"   
 [21] "cpp11"         "crayon"        "credentials"   "curl"         
 [25] "data.table"    "DBI"           "dbplyr"        "desc"         
 [29] "devtools"      "diffobj"       "digest"        "dplyr"        
 [33] "dtplyr"        "ellipsis"      "evaluate"      "fansi"        
 [37] "farver"        "fastmap"       "forcats"       "fs"           
 [41] "gargle"        "generics"      "gert"          "ggplot2"      
 [45] "gh"            "gitcreds"      "glue"          "googledrive"  
 [49] "googlesheets4" "gtable"        "haven"         "highr"        
 [53] "hms"           "htmltools"     "httr"          "ids"          
 [57] "ini"           "isoband"       "jsonlite"      "knitr"        
 [61] "labeling"      "lifecycle"     "lubridate"     "magrittr"     
 [65] "markdown"      "memoise"       "mime"          "modelr"       
 [69] "munsell"       "openssl"       "pillar"        "pkgbuild"     
 [73] "pkgconfig"     "pkgload"       "praise"        "prettyunits"  
 [77] "processx"      "progress"      "ps"            "purrr"        
 [81] "R6"            "rappdirs"      "rcmdcheck"     "RColorBrewer" 
 [85] "Rcpp"          "readr"         "readxl"        "rematch"      
 [89] "rematch2"      "remotes"       "reprex"        "rlang"        
 [93] "rmarkdown"     "roxygen2"      "rprojroot"     "rstudioapi"   
 [97] "rversions"     "rvest"         "scales"        "selectr"      
[101] "sessioninfo"   "stringi"       "stringr"       "svglite"      
[105] "sys"           "systemfonts"   "testthat"      "tibble"       
[109] "tidyr"         "tidyselect"    "tidyverse"     "tinytex"      
[113] "usethis"       "utf8"          "uuid"          "vctrs"        
[117] "viridisLite"   "waldo"         "whisker"       "withr"        
[121] "xfun"          "xml2"          "xopen"         "yaml"         
[125] "zip"           "base"          "boot"          "class"        
[129] "cluster"       "codetools"     "compiler"      "datasets"     
[133] "foreign"       "graphics"      "grDevices"     "grid"         
[137] "KernSmooth"    "lattice"       "MASS"          "Matrix"       
[141] "methods"       "mgcv"          "nlme"          "nnet"         
[145] "parallel"      "rpart"         "spatial"       "splines"      
[149] "stats"         "stats4"        "survival"      "tcltk"        
[153] "tools"         "utils"

In [ ]:
# Install system dependencies
# For sf
system("apt-get -y update && apt-get install -y libudunits2-dev libgdal-dev libgeos-dev libproj-dev")
# For magick and fftwtools
system("apt-get install -y libmagick++-dev libfftw3-dev")

Usage of the installed packages in this tutorial:

* `BiocManager`: Install packages from Bioconductor
* `biomaRt`: Query biomart to map Ensembl gene IDs to gene symbols and get genome annotations
* `BUSpaRse`: Read kallisto bustools output into R as sparse matrix and plot scRNA-seq knee plot
* `EBImage`: Segment H&E image to detect area covered by tissue
* `DropletUtils`: Read Cell Ranger h5 when using Cell Ranger matrices provided by the authors.
* `SpatialExperiment`: Store Visium data, with the gene count matrix, spot coordinates, and the H&E image in R 
* `UpSetR`: Make upset plots, an alternative to the Venn diagram suitable for larger number of sets
* `spatialLIBD`: Plot gene expression and spot level metadata on top of H&E image
* `automap`: Fit variogram models (will explain later in the tutorial)
* `spdep`: Compute spatial autocorrelation metrics such as Moran's I
* `scran`: scRNA-seq style data normalization and building k nearest neighbor graphs for clustering
* `scater`: scRNA-seq style quality control (QC) and exploratory data analysis (EDA) plots
* `igraph`: Louvain clustering
* `dittoSeq`: Colorblind friendly palettes for scRNA-seq
* `BayesSpace`: Spatially informed clustering of Visium spots
* `ggforce`: Make alluvial plot to compare non-spatial and spatially informed clustering, and plot deconvolved cell type composition on spot coordinates as mini pie charts
* `SPARK`: Spatially variable genes
* `sparseMatrixStats`: Compute mean gene expression profile in each cluster of gene expression of patterns
* `RCTD`: Cell type deconvolution of Visium spots


In [ ]:
# CRAN
system.time({
  install.packages(c("BiocManager", "automap", "UpSetR", "ggforce", "spdep",
                     "igraph", "hexbin", "sctransform", "uwot", "patchwork"), 
                   Ncpus = 2)
  # Bioconductor
  BiocManager::install(c("SpatialExperiment", "biomaRt", "BUSpaRse", "EBImage", 
                         "sparseMatrixStats", "BayesSpace", "spatialLIBD",
                         "scran", "scater", "DropletUtils", "dittoSeq"), 
                       Ncpus = 2)
  # From GitHub
  remotes::install_github("xzhoulab/SPARK", Ncpus = 2)
  remotes::install_github("dmcable/RCTD", Ncpus = 2)
})

Download the chicken genome and annotation from Ensembl to build the kallisto index. This takes a while, depending on internet.

In [ ]:
# So the download doesn't time out
options(timeout = Inf)

In [ ]:
dir.create("reference")
system.time({
  # Genome
  download.file("http://ftp.ensembl.org/pub/release-104/fasta/gallus_gallus/dna/Gallus_gallus.GRCg6a.dna_rm.toplevel.fa.gz",
                destfile = "reference/Gallus_gallus.GRCg6a.dna_rm.toplevel.fa.gz",
                method = "wget")
  # GTF
  download.file("http://ftp.ensembl.org/pub/release-104/gtf/gallus_gallus/Gallus_gallus.GRCg6a.104.chr.gtf.gz",
                destfile = "reference/Gallus_gallus.GRCg6a.104.chr.gtf.gz",
                method = "wget")
})

The genome, with masks for repetitve regions that might impede alignment, and the genome annotation with only the canonical chromosomes, have been downloaded into the `reference` directory. Then we need to change the permission of those files in order to use them to build a kallisto index; this issue is peculiar to Google Colab.

In [ ]:
system("chmod -R 777 reference")

Download Space Ranger metadata and low resolution H&E images. These are required by SpatialExperiment and Seurat when loading the Visium data into R as a SpatialExperiment or Seurat object and plot gene expression in space on top of the H&E image. 

In [ ]:
dir.create("data")
download.file("https://caltech.box.com/public/static/ffwz74y63jwijok0tljbn11xqsxvlok6",
              destfile = "data/tissue_lowres_image.png", method = "wget")
download.file("https://caltech.box.com/public/static/azhrdbc4784otpxklfsvfh4uwmexrubn",
              destfile = "data/scalefactors_json.json", method = "wget")
download.file("https://caltech.box.com/public/static/wg0i5qwaq2fw2m2s1fy5pndiv0tcxx7a",
              destfile = "data/tissue_positions_list.csv", method = "wget")

Download Space Ranger filtered matrix to compare to kallisto bustools results

In [ ]:
dir.create("their")
download.file("https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM4502485&format=file&file=GSM4502485%5Fchicken%5Fheart%5Fspatial%5FRNAseq%5FD14%5Ffiltered%5Ffeature%5Fbc%5Fmatrix%2Eh5",
              destfile = "their/D14.h5", method = "wget")

Later, we will do cell type deconvolution of the Visium spots, using the D14 left vintricle (LV) and right ventricle (RV) scRNA-seq datasets from the same study as reference. As this tutorial is about Visium and other tutorials on this website already cover scRNA-seq fastq file processing and EDA, we will use the filtered Cell Ranger matrices provided by the authors.

In [ ]:
download.file("https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4502nnn/GSM4502480/suppl/GSM4502480_chicken_heart_scRNAseq_D14_LV_filtered_feature_bc_matrix.h5", 
              destfile = "their/D14_LV.h5", method = "wget", quiet = TRUE)
download.file("https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4502nnn/GSM4502481/suppl/GSM4502481_chicken_heart_scRNAseq_D14_RV_filtered_feature_bc_matrix.h5",
              destfile = "their/D14_RV.h5", method = "wget", quiet = TRUE)

In [ ]:
library(tidyverse)
library(gstat)
library(automap)
library(spdep)
library(SpatialExperiment)
library(spatialLIBD)
library(scater)
library(scran)
library(dittoSeq)
library(sctransform)
library(DropletUtils)
library(BUSpaRse)
library(igraph)
library(jsonlite)
library(UpSetR)
library(biomaRt)
library(EBImage)
library(BayesSpace)
library(ggforce)
library(patchwork)
library(SPARK)
library(sparseMatrixStats)
library(RCTD)
library(scales)
theme_set(theme_bw() + 
            theme(axis.text = element_text(size = 12),
                  axis.title = element_text(size = 14),
                  legend.title = element_text(size = 14)))

# From fastq files to gene count matrix

Build the kallisto index from the references; this takes a few minutes. Output of `system.time()` is in seconds and the `elapsed` field is the amount of time running the command took.

In [ ]:
(cmd_ref <- paste("cd reference && kb ref -i index.idx -g tr2g.txt", 
            "-f1 transcriptome.fa Gallus_gallus.GRCg6a.dna_rm.toplevel.fa.gz", 
            "Gallus_gallus.GRCg6a.104.chr.gtf.gz"))

In [ ]:
system.time(system(cmd_ref))

The fastq files can be streamed into kallisto bustools. The `-x` flag indicates technology, which really means a specification of barcode, UMI, and read positions and lengths within the fastq files. For Visium, this specification is the same as that of 10X v3, so `10xv3` is used here. However, Visium has a different set of barcode whitelist from 10X v3 that comes with Space Ranger. The whitelist file has been extracted from Space Ranger and can be downloaded here:

In [ ]:
download.file("https://caltech.box.com/public/static/ry3bptqnsijglq64q37t1rorur209kxy",
              destfile = "reference/visium-v1.txt", method = "wget")

The data can be downloaded from NCBI's GEO, which requires `fasterq-dump`, which doesn't work on Colab, but the fastq files can be directly downloaded from the European Nucleotide Archive (ENA), with the following URLs:

In [ ]:
(urls <- paste0("http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR116/066/SRR11616466/SRR11616466_", 
               1:2, ".fastq.gz"))

However, in this case, this doesn't work for Colab either, because the data has to be transmitted from the UK to California where this tutorial was written and download is very slow. It would take hours to download the fastq files this way or stream them into kallisto bustools, and Colab is meant for interactive use. We downloaded the fastq files and shared them on Box to speed up your download when running this notebook. Here the fastq files are streamed into kallisto bustools.

In [ ]:
urls2 <- c("https://caltech.box.com/public/static/u8uy07lm7mosdlo4o111btc21344ebva",
           "https://caltech.box.com/public/static/1sopw3qxlfrx9t5ekr35cb8lkpouj294")

In [ ]:
(cmd_count <- paste("kb count -i reference/index.idx -g reference/tr2g.txt", 
                   "-x 10xv3 -o output -t2 -m 10G -w reference/visium-v1.txt",
                   paste(urls2, collapse = " ")))

In [ ]:
system.time(system(cmd_count))

It took 20 something minutes, which is a lot faster than Cell Ranger.

# Remove empty spots
In scRNA-seq, the knee plot is commonly used to identify and filter empty droplets. In contrast, in Visium, filtering empty spots is easier, because once the spots are aligned to the H&E image, which spot is on the tissue is known. 

In [ ]:
# Read the Visium gene count matrix
D14_unfiltered <- read_count_output("output/counts_unfiltered", name = "cells_x_genes")

In [ ]:
dim(D14_unfiltered)

The 4992 columns are all the 4992 spots in each tissue area on the Visium slide. Space Ranger aligns the spots to the H&E image, segments the H&E image to detect areas covered by tissue, and gives coordinates of spots in array rows and columns and pixels in the full resolution H&E image. The coordinates and whether Space Ranger thinks each spot is on the tissue (`in_tissue` column) is in the `tissue_positions_list.csv` file in Space Ranger output.

In [ ]:
cns <- c("barcode", "in_tissue", "array_row", "array_col",
         "pxl_row_in_fullres", "pxl_col_in_fullres")
tissue_positions <- read_csv("data/tissue_positions_list.csv", col_names = cns)
head(tissue_positions)

Which spots does Space Ranger think are on the tissue? Here `options(repr.plot.width=12, repr.plot.height=8)` sets figure size in Jupyter Notebook's R kernel. In RStudio, with R Markdown, use `fig.width = 12, fig.height = 8` in the chunk options to set figure size.

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
ggplot(tissue_positions, aes(pxl_col_in_fullres, pxl_row_in_fullres, 
                             color = as.character(in_tissue))) +
  geom_point(size = 2) +
  scale_color_brewer(palette = "Set2", name = "in_tissue") +
  coord_equal()

How many in tissue spots does Space Ranger think there are?

In [ ]:
sum(tissue_positions$in_tissue > 0)

What does the H&E image look like? 



In [ ]:
img <- readImage("data/tissue_lowres_image.png")
display(img, "raster")

Space ranger considers spots in the empty spaces inside ventricles and atria as covering tissue, and we will use `EBImage` to segment the image and remove those spots. The authors of the publication from which this dataset was taken seemed to have used those spots as if they are in tissue.

Furthermore, `ggplot2` seems to plot the tissue up side down, because `ggplot2` uses the Cartesian coordinates by default, in which small values of y are at the bottom, while in images, small values of y are at the top. When plotting with `spatialLIBD` or Seurat, the spots will be aligned to the image and the orientation of the image will be used.

In [ ]:
# Convert to grayscale
img <- channel(img, "gray")
# histogram
hist(img)

This image is easy to threshold for the background is even. We can threshold it and remove some debris with an [opening operation](https://docs.opencv.org/master/d9/d61/tutorial_py_morphological_ops.html). The peak at highest intensity is the white background, and 0.9 seems like a good threshold as it's above the peak that is the tissue.

Opening is erosion followed by dilation. In the erosion morphological operation of a binary image such as the tissue mask indicating which pixel is in the tissue, a kernel, such as a disk, is scanned across the image; if any pixel covered by the kernel is 0, then all pixels covered will be set to 0, so small areas of 1s (which often is noise) and edges of large areas of 1s will be removed. Dilation does the opposite of erosion; if any pixel covered by the kernel is 1, then all pixels covered will be set to 1, so areas of 1s will be dilated.

In [ ]:
# Thresholding
mask <- img < 0.9
display(mask, "raster")

In [ ]:
# Kernel for opening
options(repr.plot.width=3, repr.plot.height=2)
kern <- makeBrush(5, shape = "disc")
display(kern, "raster", interpolate = FALSE)

In [ ]:
# Mask after opening
options(repr.plot.width=12, repr.plot.height=8)
mask2 <- opening(mask, kern)
display(mask2, "raster")

After the opening operation, the tissue mask looks cleaner and the debris is gone. We can then convert pixels in the low resolution image to coordinates in the full resolution image as used in SpatialExperiment; the full resolution image is not available for download. 

Such conversion can be done as the `scalefactors_json.json` specifies scale factors between the low resolution and full resolution images, as well as the number of pixels per spot in the full resolution image.

In [ ]:
(sfs <- read_json("data/scalefactors_json.json", simplifyVector = TRUE))

Multiplying the `tissue_lowres_scalef` field with the full resolution dimension will get the low resolution dimension.

In [ ]:
# Get low resolution pixel coordinates
tissue_positions <- tissue_positions %>% 
  mutate(pxl_row_in_lowres = round(pxl_row_in_fullres * sfs$tissue_lowres_scalef),
         pxl_col_in_lowres = round(pxl_col_in_fullres * sfs$tissue_lowres_scalef))

In [ ]:
# I think the col and row indexing is 0 based
# Get the mask in data frame form to join with tissue_position and filter
mask_df <- tibble(x = rep(seq_len(nrow(mask2))-1, ncol(mask2)),
                  y = rep(seq_len(ncol(mask2))-1, each = nrow(mask2)))
mask_df$in_tissue2 <- as.vector(mask2)

The mask based on `EBImage` segmentation in data frame form is joined to the `tissue_positions` data frame, so a new column will indicate whether each Visium spot is in the tissue based on the new segmentation. 

In [ ]:
tissue_positions <- left_join(tissue_positions, mask_df, 
                              by = c("pxl_row_in_lowres" = "y",
                                     "pxl_col_in_lowres" = "x"))

These are the spots deemed to be in the tissue based on the new segmentation.

In [ ]:
ggplot(tissue_positions, aes(pxl_col_in_lowres, pxl_row_in_lowres, 
                             color = in_tissue2)) +
  geom_point(size = 2) +
  scale_color_brewer(palette = "Set2") +
  coord_equal()

So it works. What proportion of spots Space Ranger thinks are in tissue are still considered in tissue?

In [ ]:
tissue_positions %>% 
  filter(in_tissue > 0) %>% 
  pull(in_tissue2) %>% mean()

The vast majority of them. What proportion of spots Space Ranger does not think are in tissue are now considered in tissue?

In [ ]:
tissue_positions %>% 
  filter(in_tissue < 1) %>% 
  pull(in_tissue2) %>% mean()

Very few. Now format `tissue_positions` and write a new `tissue_positions_list.csv`. The columns must be in this order:

In [ ]:
cns

And the `in_tissue` column must be numeric rather than logical. The `tissue_positions` data frame is then written to a csv file without headers. The format is required for SpatialExperiment and Seurat to read the spot coordinates properly.

In [ ]:
tissue_positions <- tissue_positions %>% 
  mutate(in_tissue2 = as.numeric((in_tissue > 0) & in_tissue2)) %>% 
  dplyr::select(-in_tissue, -contains("lowres")) %>% 
  dplyr::rename(in_tissue = in_tissue2)

In [ ]:
# Remove the "-1" in the barcode from Space Ranger, which is unnecessary here
tissue_positions <- tissue_positions %>% 
  mutate(barcode = str_remove(barcode, "-1$"))

In [ ]:
# Backup the original
file.rename("data/tissue_positions_list.csv", "data/tissue_positions_list1.csv")
write_csv(tissue_positions[,cns], "data/tissue_positions_list.csv",
          col_names = FALSE)

In [ ]:
D14_filtered <- D14_unfiltered[, tissue_positions$barcode[tissue_positions$in_tissue > 0]]
D14_filtered <- D14_filtered[rowSums(D14_filtered) > 0,]
dim(D14_filtered)

There are 1622 spots left after removing the spots in the empty spaces inside the atria and ventricles.

## Explore all spots
As there are "only" 4992 spots in total, it's easy to plot them and see what the empty spots are like in terms of common scRNA-seq QC metrics such as total UMI counts per barcode (i.e. spot here), number of genes detected per barcode, and percentage of mitochondrially encoded UMI counts. What's different here is that we unequivocably know which spots are empty, and with full resolution image, we know the proportion of each spot covered by tissue. Furthermore, because the spots are arranged in space and transcripts can diffuse out of the tissue, empty spots aren't necessarily zero in those QC metrics; values of these metrics might depend on distance to the tissue.

First, as the gene count matrix's row names are unique Ensembl IDs that are not human readable, let's get the corresponding gene symbols and some genome annotations from Ensembl biomart. 

In [ ]:
mart <- useEnsembl("ensembl", species2dataset("Gallus gallus"))
gns <- getBM(c("ensembl_gene_id", "external_gene_name", "gene_biotype",
               "chromosome_name"), 
             mart = mart)

In [ ]:
# Remove undetected genes
D14_unfiltered <- D14_unfiltered[rowSums(D14_unfiltered) > 0,]
rownames(D14_unfiltered) <- str_remove(rownames(D14_unfiltered), "\\.\\d+$")

After removing the undetected genes, what's the proportion of non-zero entries?

In [ ]:
prop_non0 <- function(dgc) {
  length(dgc@x) / (nrow(dgc) * ncol(dgc))
}

In [ ]:
prop_non0(D14_unfiltered)

With all the empty spots, of course the matrix is quite sparse, with only about 7.5% of entries that are not 0. 

Then we put the matrix and info about genes into a `SpatialExperiment` object, which inherits from `SingleCellExperiment`, and which facilitates plotting and gives access to dimension reduction and differential expression (DE) methods for `SingleCellExperiment`. `Seurat` can also be used for Visium data, but we use `SpatialExperiment` here because the spatially informed clustering method `BayesSpace` used in this tutorial requires `SingleCellExperiment`. 

In [ ]:
# Row or gene metadata
rd <- gns[match(rownames(D14_unfiltered), gns$ensembl_gene_id),]
names(rd) <- c("gene_search", "symbol", "gene_biotype", "chromosome")

In [ ]:
# Make sure barcodes in tissue_positions are in the same order as in the matrix
tissue_positions <- tissue_positions[match(colnames(D14_unfiltered), 
                                           tissue_positions$barcode),]

In [ ]:
# Column or spot metadata
cd <- as.data.frame(tissue_positions[,cns])
rownames(cd) <- cd$barcode
# These column names are required by BayesSpace
names(cd) <- c("spot", "in_tissue", "row", "col", "imagerow", "imagecol")
cd$spot <- NULL

In [ ]:
# Spatial data, rename columns or the plots will be transposed; bug in spatialLIBD
sd <- setNames(tissue_positions[, c("pxl_row_in_fullres", "pxl_col_in_fullres")], 
               c("pxl_col_in_fullres", "pxl_row_in_fullres"))

`SpatialExperiment` differs from `SingleCellExperiment` in that the low resolution H&E image is stored within the object for plotting, and the spatial coordinates of the spots are stored in `spatialData`, which is distinct from `colData` of `SingleCellExperiment`. The H&E image and metadata from Space Ranger are read by the `readImgData()` function. Otherwise `SpatialExperiment` behaves like `SingleCellExperiment`.

In [ ]:
img <- readImgData("data", sample_id = "D14")

In [ ]:
spe <- SpatialExperiment(assays = list(counts = D14_unfiltered),
                         colData = DataFrame(cd), rowData = DataFrame(rd),
                         spatialData = DataFrame(sd),
                         spatialCoordsNames = c("pxl_row_in_fullres", "pxl_col_in_fullres"),
                         imgData = img, sample_id = "D14")

In [ ]:
colData(spe)$nCounts <- colSums(D14_unfiltered)
colData(spe)$nGenes <- colSums(D14_unfiltered > 0)

How is total UMI counts per spot distributed in space?

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe, "D14", "nCounts", assayname = "counts")

There's a area outside the atria with high total counts. Also, there seems to be a halo around the tissue, which indicates diffusion of transcripts out of the tissue. There's an R package that can correct for transcript diffusion in Visium, [SpotClean](https://github.com/zijianni/SpotClean). Moreover, there appears to be spatial autocorrelation in total counts, which seems to be related to biological regions (e.g. higher total counts at the valves), so I doubt if total counts should be corrected for as a technical artefact as in some scRNA-seq data normalization methods using size factors. It might be biologically relevant.

According to the [Visium library preparation protocol](https://assets.ctfassets.net/an68im79xiti/3GGIfH3RWpd1bFVha1pexR/8baa08d9007157592b65b2cdc7130990/CG000239_VisiumSpatialGeneExpression_UserGuide_RevD.pdf), reverse transcription and second strand synthesis are performed on the slide, while cDNA amplification is performed after the cDNA sample is removed from the slide. Technical variation that is spatially relevant would occur on the slide. A potential source of such technical variation may be amount of barcodes, UMIs, and capture sequences printed on each spot. I don't know how Visium spots are printed, but most likely they are printed row by row or column by column, and if so, spatial autocorrelation of amount of barcodes printed would be along the rows or columns rather than in both x and y directions as seen here for total counts. Another potential source of spatial technical variation is fluid handling during reverse transcription and second strand synthesis on the slide; exploring spatial autocorrelation in empty spots across datasets may shed some light on this. However, this does not explain how total counts relate to biological regions as seen here.

Plot the violin plot of total counts as in scRNA-seq.

In [ ]:
colData(spe)$in_tissue3 <- colData(spe)$in_tissue > 0

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
plotColData(spe, "nCounts", "in_tissue3") + 
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14))

Spots not in tissue generally have low total counts, except a small number that is the halo around the tissue and the area outside the atria. How about number of genes detected per spot?

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe, "D14", "nGenes", assayname = "counts")

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
plotColData(spe, "nGenes", "in_tissue3") + 
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14))

Same pattern as total counts. Having about 2000 genes and 10000 UMI counts per barcode is similar to or somewhat higher than in some 10X v3 datasets, though because each Visium spot has diameter of 55 $\mu$m while a cell is about 10 $\mu$m across, each spot can have multiple cells. 

In scRNA-seq, percentage of mitochondrially encoded counts is often used to remove low quality cells, because the mitochondria are enclosed in a double membrane, so when the cell bursts, mitochondrial transcripts are less likely to be lost than cytosolic transcripts, leading to higher percentage of mitochondrial counts in cells that have burst. How relevant is percentage of mitochondrial counts to Visium?

In [ ]:
# Compute percentage of mitochondrial counts
mt_genes <- rowData(spe)$chromosome == "MT"
colData(spe)$pct_mt <- colSums(D14_unfiltered[mt_genes,]) / colData(spe)$nCounts * 100

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe, "D14", "pct_mt", assayname = "counts")

The empty spots often have high proportion of mitochondrial counts, though the proportion is still high in some spots covered by tissue. This is the heart, and cardiomyocytes have a lot of mitochondria, so this isn't surprising. The pattern also seems biologically relevant, as percentage of mitochondrial counts is lower in the valves. There's also clearly spatial autocorrelation. Furthermore, as the tissue is sectioned, some cells are inevitably sliced through, releasing the mitochondria.

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
plotColData(spe, "pct_mt", x = "in_tissue3") + 
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14))

Spots on and off the tissue can both have pretty high percentage of mitochondrial counts. 

## scRNA-seq style filtering
Just curious, what if we filter the spots with the knee plot like in non-spatial scRNA-seq?

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
df <- get_knee_df(D14_unfiltered)
inflection <- get_inflection(df, lower = 500)
knee_plot(df, inflection)

The number of spots after this kind of filtering (2187) is somewhat higher than the number of spots Space Ranger deems to be on the tissue, which is 1967, and much higher than the number of spots on tissue determined by image segmentation, which is 1622. Where are the knee plot filtered spots in space?

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe[, colData(spe)$nCounts > inflection], "D14", "nCounts", 
         assayname = "counts")

Mostly on tissue, but it includes a halo around the tissue and the area inside atria and ventricles.

# EDA of filtered matrix
From this point on, we will use the image segmentation based filtering and focus on the spots that are on tissue.

In [ ]:
D14_filtered <- D14_unfiltered[, tissue_positions$in_tissue > 0]
D14_filtered <- D14_filtered[rowSums(D14_filtered) > 0,]

In [ ]:
spe2 <- spe[rownames(D14_filtered), colnames(D14_filtered)]
dim(spe2)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe2, "D14", "nCounts", assayname = "counts")

How does total count relate to number of genes detected?

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
colData(spe2) %>% 
  as.data.frame() %>% 
  ggplot(aes(nCounts, nGenes)) +
  geom_point(alpha = 0.3) +
  labs(x = "Total counts per spot", y = "Number of genes detected")

This pattern is similar to that of scRNA-seq datasets. In scRNA-seq, after the knee plot filtering, percentage of mitochondrial counts is used to remove low quality cells. Usually cells with high percentage of mitochondrial counts also have low total counts. After removing the empty spots, how does percentage of mitochondrial counts relate to total counts?

In [ ]:
colData(spe2) %>% 
  as.data.frame() %>% 
  ggplot(aes(nCounts, pct_mt)) +
  geom_point(alpha = 0.3) +
  labs(x = "Total counts per spot", y = "Percentage mitochondrial counts")

This pattern is quite different from that commonly seen in scRNA-seq datasets; there aren't spots with low total counts and distinctively high percentage of mitochondrial counts. Are there low quality spots that are in tissue? Perhaps. Perhaps damaged regions of tissue can be discerned from the H&E image, though perhaps a higher resolution image is more helpful. 

Single cell RNA-seq data is overdispersed compared to a Poisson distribution, with variance higher than mean especially for genes with higher means. This can be visualized by plotting mean and variance of each gene across cells or spots in the dataset, or mean and squared coefficient of variance (CV2, variance divided by mean squared). In a Poisson distribution, the mean and the variance are both $\lambda$, and CV2 is $1/\lambda$.

In [ ]:
rowData(spe2)$mean <- rowMeans(counts(spe2))
rowData(spe2)$var <- rowVars(counts(spe2))
rowData(spe2)$cv2 <- rowData(spe2)$var/rowData(spe2)$mean^2

The red line here of $y = x$ indicates the theoretical trend if the gene expression is Poisson distributed. Many more highly expressed genes are above this line, which means that their variances are higher than expected under Poisson distribution.

In [ ]:
rowData(spe2) %>% 
  as.data.frame() %>% 
  ggplot(aes(mean, var)) +
  geom_hex(bins = 50) +
  scale_fill_viridis_c() +
  geom_abline(slope = 1, intercept = 0, color = "red") +
  scale_x_log10() +
  scale_y_log10() +
  annotation_logticks() +
  coord_equal()

The red line here of $y = 1/x$ (x and y are both on log scale here) is the theoretical trend if the gene expression is Poisson distributed. Again, many more highly expressed genes are above this line.

In [ ]:
rowData(spe2) %>% 
  as.data.frame() %>% 
  ggplot() +
  geom_hex(aes(mean, cv2), bins = 50) +
  geom_function(fun = function(x) 1/x, color = "red") +
  scale_fill_viridis_c() +
  scale_x_log10() +
  scale_y_log10() +
  annotation_logticks() +
  coord_equal()

This pattern is similar to that in scRNA-seq datasets.

## Spatial autocorrelation
Spatial coordinates of the spots opened up a world of EDA that is unavailable to non-spatial scRNA-seq, and EDA methods in spatial statistics, i.e. exploratory spatial data analysis (ESDA), usually applied to geography, can be used. Among the things looked at in ESDA is spatial autocorrelation.

Tobler's first law of geography says,

> Everything is related to everything else, but near things are more related than distant things.

This is spatial autocorrelation, which we experience in daily life. For instance, weather in nearby cities tend to be more similar than weather in cities that are further apart. Here we use the classic R packages `spdep` and `gstat` to explore spatial autocorrelation in this Visium dataset. These two R packages date back to the early 2000s, well before the advent of Tidyverse. In fact, `gstat` is about as old as R itself, started in 1993 as an S package and went on CRAN in 2003. These old package use `sp`, which is the forerunner of Tidyverse `sf`. So we begin with constructing a `SpatialPointsDataFrame` from `sp` with the spot coordinates and variables whose spatial autocorrelation to explore. Here we mainly show spatial autocorrelation of total counts per spot, but such analyses can be done to any spot level metadata or gene. It's infeasible to manually inspect these plots for all 20,000 genes, but as we shall see later in this tutorial, some methods to find spatially variable genes rely on spatial autocorrelation.


In [ ]:
# Get data frame of spot metadata and coordinates
spdf <- as.data.frame(colData(spe2))

The spot coordinates are in pixels in the full resolution H&E image. Here, to make the correlograms and variograms -- which can show length scales of spatial autocorrelation -- more interpretable, we convert the coordinates from pixels into microns. The `scalefactors_json.json` file from Space Ranger contains info of the number of pixels across each spot. Also, according to the [10X website](https://kb.10xgenomics.com/hc/en-us/articles/360035487812-What-is-the-size-of-the-spots-on-the-Visium-Gene-Expression-Slide-), the spots have diameter 55 $\mu$m. Then we have enough info to convert pixels to microns. Also, we scale the total counts and number of genes detected to avoid numeric issues with the variogram without affecting spatial autocorrelation.

In [ ]:
sfs <- read_json("data/scalefactors_json.json", simplifyVector = TRUE)

In [ ]:
spdf$x <- spdf$imagecol/sfs$spot_diameter_fullres * 55
spdf$y <- spdf$imagerow/sfs$spot_diameter_fullres * 55
spdf$nCounts <- spdf$nCounts / 1000
spdf$nGenes <- spdf$nGenes / 1000

This will convert a regular data frame into a `SpatialPointsDataFrame`.

In [ ]:
coordinates(spdf) <- ~ x + y

Check that the coordinates in the `SpatialPointsDataFrame` are correct

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
plot(spdf, pch = 19, col = "gray")

### Moran's I
There are several ways to quantify spatial autocorrelation, the most common of which is Moran's I:

$$
I = \frac{n}{\sum_{i=1}^n \sum_{j=1}^n w_{ij}} \frac{\sum_{i=1}^n \sum_{j=1}^n w_{ij} (x_i - \bar{x})(x_j - \bar{x})}{\sum_{i=1}^n (x_i - \bar{x})^2}
$$

Where $n$ is the number of spots or locations, $i$ and $j$ are different locations, or spots in the Visium context, $x$ is a variable with values at each location, and $w_{ij}$ is a spatial weight, which can be inversely proportional to distance between spots or an indicator of whether two spots are neighbors, subject to various definitions of neighborhood and whether to normalize the number of neighbors. The `spdep` package uses the neighborhood. 

Moran's I takes values between -1 and 1. For positive spatial autocorrelation, i.e. nearby spots tend to be more similar, Moran's I will be positive. For negative spatial autocorrelation, i.e. nearby spots tend to be more dissimilar, Moran's I will be negative. When the variable is distributed in space randomly like salt and pepper, then Moran's I will be around 0. Positive Moran's I indicates global structure, while negative Moran's I indicates local structure.

In Visium, spots are arranged in a hexagonal grid, so the problem of whether diagonal neighbors as in a square grid, i.e. whether to use queen (with diagonal neighbors or neighbors sharing border vertices) or rook (without, or only with neighbors sharing border lines) is irrelevant. According to [10X's website](https://kb.10xgenomics.com/hc/en-us/articles/360035487572-What-is-the-spatial-resolution-and-configuration-of-the-capture-area-of-the-Visium-Gene-Expression-Slide-), the spots are 100 $\mu$m apart center to center, so distance between spots can be used to determine if spots are adjacent. In practice, after converting pixels to microns above, adjacent spots are 80 something microns apart, but a range of distances can still be used to identify adjacent spots. Perhaps the spots are actually smaller than 55 $\mu$m.

Here two spots are neighbors if they are adjacent, which is determined with `dnearneigh()` where the `d` stands for distance, with minimum distance of 50 microns and maximum distance of 100 microns. The `nb2listw()` function returns a list of neighbors of each spot, and `style = "W"` means each neighbor of a spot has a weight of 1 divided by the number of neighbors of the spot of interest. The default is `style = "W"`. One spot here is not adjacent to any other spot; `zero.policy = TRUE` sets that entry in the neighborhood list to an empty list rather than throwing an error.

In [ ]:
spot_nb <- dnearneigh(spdf, 50, 100)
spot_w <- nb2listw(spot_nb, style = "W", zero.policy = TRUE)

If the variable is normally distributed, then there's a parametric test to see whether Moran's I is higher or lower than expected by chance if the variable is not actually spatially autocorrelated. However, here the total counts don't seem normally distributed, so we use a permutation test, in which the total counts per spot are permuted among the locations and Moran's I is calculated for each of the permutation.

In [ ]:
options(repr.plot.width=9, repr.plot.height=6)
set.seed(29)
ncounts_mc <- moran.mc(spdf$nCounts, spot_w, nsim = 200, zero.policy = TRUE)
plot(ncounts_mc, cex.lab = 1.7, cex.axis = 1.5, cex.main = 1.7, cex.sub = 1.3)

Here the actual Moran's I is way higher than expected from permutation. Moran's I for permuted total counts is around 0, which means no spatial autocorrelation. It's reasonable to say that the spatial autocorrelation in total counts is real. 

Just now, the first order neighbor, i.e. adjacent spots, is used for the spatial weights in Moran's I. However, higher order neighbors can also be used, e.g. neighbors of neighbors, to explore spatial scale of autocorrelation. Because of the regular Visium hexagonal grid, order of neighbors is a proxy of physical distance.

In [ ]:
ncounts_corr_I <- sp.correlogram(spot_nb, spdf$nCounts, order = 10, method = "I",
                                 zero.policy = TRUE)
plot(ncounts_corr_I)

The error bars are +/- twice the standard deviation of the estimated Moran's I. Here spatial autocorrelation diminishes until 5 spots away, then it becomes slightly negative, suggesting that this is the scale of local structures. Total counts per spot is plotted in space again to compare to the Moran's I correlogram.

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe2, "D14", "nCounts", assayname = "counts")

Another spatial autocorrelation metric is Geary's C, defined as:

$$
C = \frac{(n-1)}{2\sum_{i=1}^n \sum_{j=1}^n w_{ij}} \frac{\sum_{i=1}^n \sum_{j=1}^n w_{ij}(x_i - x_j)^2}{{\sum_{i=1}^n (x_i - \bar{x})^2}}
$$

Geary's C well below 1 indicates positive spatial autocorrelation, and above 1 indicates negative spatial autocorrelation.

In [ ]:
options(repr.plot.width=9, repr.plot.height=6)
ncounts_corr_C <- sp.correlogram(spot_nb, spdf$nCounts, order = 10, method = "C",
                                 zero.policy = TRUE)
plot(ncounts_corr_C)

The Geary's C correlogram recapitulates the pattern in the Moran's I correlogram. Moran's I can also be calculated locally, at each spot:

$$
I_i = \frac{x_i - \bar{x}}{\sum_{k=1}^n (x_k - \bar{x})^2/(n-1)} \sum_{j=1}^n w_{ij} (x_j-\bar{x})
$$

A positive value means a spot is similar to its neighbors in a variable, which means a hotspot or cold spot. A negative value means the spot is dissimilar to its neighbors, which may mean an outlier. Local Moran's I can be calculated with `spdep::localmoran()`.

Related to local Moran's I is the Moran plot, which plots values at each spot against its lagged values, i.e. weighted sum of values from its neighbors. Spots with high value with neighbors that also have high values are in hot spots, while spots with low value with neighbors that also have low values are in cold spots.

In [ ]:
mp_ncounts <- moran.plot(spdf$nCounts, spot_w, zero.policy = TRUE, 
                         cex.lab = 1.7, cex.axis = 1.5, cex.main = 1.7)

The circled point is a spot that is not adjacent to any other spot. The dashed lines are the means of total counts and the lagged values. The labeled points are influential to fitting the linear model that is the solid line. Where are the hot spots of total counts?

In [ ]:
mean_x <- mean(mp_ncounts$x)
mean_wx <- mean(mp_ncounts$wx) # Lagged

In [ ]:
colData(spe2)$ncounts_hot <- mp_ncounts$x > mean_x & mp_ncounts$wx > mean_wx & mp_ncounts$is_inf

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)
vis_clus(spe2, "D14", "ncounts_hot")

For total cocunts, there are hot spots at the valves and the septum. That could be due to larger number of cells per spot, or cells hving more transcripts, or some technical effects that don't affect different cell types equally, i.e. a combination of biological and technical effects. Without a high or full resolution H&E image, we cannot segment nuclei and find the number of cells per spot. 

A more commonly used way to detect hot spots and cold spots relevant to epidemiology (e.g. [a spatial analysis of COVID-19 in New York and Chicago](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7371785/)), is the [Getis-Ord local $G_i^*$](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/h-how-hot-spot-analysis-getis-ord-gi-spatial-stati.htm), calculated at each spot, which can be done in R with `spdep::localG()`. Positive values mean hot spot, while negative values mean cold spot. 

Now the question is how hot and cold spots are relevant to spatial transcriptomics. Perhaps when many genes are combined, such as as eigengenes, hot and cold spots can help identify spatial regions within tissue informed by the transcriptome. Moran's I, Geary's C, and Getis-Ord $G_i^*$ are used in a recent R package [`spatialGE`](https://doi.org/10.1101/2021.07.27.454023) to identify spatially variable genes and hotspots of expression of select marker genes in a melanoma ST dataset. 

### Variogram
A common type of analysis in spatial statistics is kriging, which means given values taken at some location in space, use a Gaussian process regression model to interpolate values where they are unavailable and give uncertainty of the interpolation. In a Gaussian process, any samples from the process follow a multivariate normal distribution, whose mean and covariance come from the mean and covariance functions that characterize the process. Often the mean is assumed to be constant, or a linear model of covariates. 

Gaussian process models are also commony used in methods that identify spatially variable genes, where the covariance function is specified by a kernel, such as a Gaussian kernel in which the covariance between spots decay exponentially with distance. Here, after modeling the covariates, the spatial component is assumed to be at least weakly stationary, i.e. with constant mean and the covariance between spots only depends on distance. However, in kriging, a more relaxed kind of stationarity is used -- instead of the covariance, the variance of differences between lagged spots, i.e. the (semi)variogram, only depends on distance between the spots. The (semi)variogram is defined as:

$$
\gamma (t) = \frac 1 2 \mathrm{Var}(x_t - x_0)
$$

Where $t$ is a spatial vector, $x_0$ is the value at a spot of interest, and $x_t$ is the value at a spatial location lagged by $t$ from the spot of interest. 

Kriging requires fitting a parametric variogram model to the data. However, how relevant is kriging to spatial transcriptomics? Perhaps it is relevant to laser capture microdissection (LCM) and GeoMX DSP data where the transcriptome is only profiled at a few select locations in tissue. In Visium, the spots already cover a regular grid in tissue, though there's some spacing between spots. Shall we interpolate gene expression between spots? Also, would Kriging help with refining resolution of Visium? How would that compare to existing methods that increase resolution of Visium, such as `BayesSpace`, and a number of deep learning methods informed by the H&E image? These are open questions that warrant new projects and cannot be addressed within this tutorial. Kriging is used in the R package `spatialGE` to interpolate cell type deconvolution scores in the melanoma ST dataset.

Whether Kriging is relevant to spatial transcriptomics or not, the variogram can be used for ESDA. A classic R package for fitting variogram models and spatial and spatiotemporal Kriging is `gstat`, which provides many different variogram models. To make fitting variograms easier, we use the package `automap`, which essentially wraps `gstat` but chooses the best fitting variogram model for us.

In [ ]:
options(repr.plot.width=9, repr.plot.height=6)
vg_fit <- autofitVariogram(nCounts ~ 1, spdf)
plot(vg_fit)

The variogram model fitted is [M. Stein's parameterization of Matern (`Ste`)](https://www.sciencedirect.com/science/article/pii/S0016706105000911). The points are the distance bins chosen by `gstat`, and the numbers at the points are number of pairs of points in the distance bin used to estimate the variogram. The nugget is the value of the variogram when distance is 0, which can be measurement error or spatial variability below the spatial resolution of the measurement. Here the semi-variance (see definition of the variogram above) is lower when the distance is small -- spatial autocorrelation, and the semi-variance rises until it levels off as spatial influence only has a certain finite range. The value of semi-variance where the variogram levels off is the sill, and the sill minus nugget is the partial sill. The range is the distance it takes for the variogram to level off. Kappa is a smoothness parameter of Matern class models. 

The variogram is useful for ESDA as it shows the range of spatial autocorrelation and values of the semi-variance. Here the total counts are scaled down by 1000, so the actual semi-variance should be scaled up by 1 million on the plot. The range seems consistent with the deminishing Moran's I shown earlier in the correlogram. In the correlogram, spatial autocorrelation deminishes at around 5 spots out, which is about 400 to 500 microns when spots are 80 something microns apart, and this is where the variogram gets more level.

Above, the variogram is isotropic, i.e. it assumes that spatial autocorrelation is the same in all directions. However, this is not necessarily the case. For example, in the cortex layers, gene expression varies more across different layers than within the same layer. This is anisotropy, where spatial autocorrelation differs in different direction. In `gstat`, the variogram can be plotted at different directions (`alpha` argument in `gstat::variogram()`), or as a map in 2D showing a grid of lags in x and y, in a variogram map, `map = TRUE`. This way we can explore anisotropy in spatial autocorrelation.

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)
ncounts_vgmap <- variogram(nCounts ~ 1, spdf, cutoff = 1000, width = 100, map = TRUE)
plot(ncounts_vgmap, threshold = 5) # Only plot when informed by at least 5 pairs of spots

It seems that the semivariance levels off more quickly along x than along y. This might be due to the structure of the heart and in this dataset the septum runs along the y axis; it would not be surprising if gene expression and cell types in the atria and the valves are quite different from those in the ventricles.

There's even more to variograms in `gstat`. For instance, covariates can be specified in the formula in `autofitVariogram()` or `variogram()`. Spatial autocorrelation can be affected by covariates. For example, in Pasadena, where this tutorial was written, the region north of the 210 freeway is quite different from the region to the south in terms of ethos and socioeconomic class; how spatial autocorrelation relates to Euclidean distance is disrupted by the freeway. In spatial transcriptomics, the covariates that affect spatial autocorrelation can be type of tissue, and if available, pathologist annotations can be used when fitting variogram models. 

What do we learn from characterizing spatial autocorrelation of total counts per spot? As I can't think of entirely technical explanations of spatial autocorrelation in total counts in the tissue, this might show something about tissue architecture not so apparent from H&E. I also wonder how spatial autocorrelation in something like total counts changes in different tissues. But more importantly, this section introduces concepts about spatial autocorrelation that can be applied to all genes and spot level metadata. It would be cool to plot correlograms, variograms, and variogram maps of all genes and see what patterns are present, as summaries of strengths and length scales of spatial autocorrelation, without having to inspect those genes by eye.

There is much more to spatial statistics than Kriging and spatial autocorrelation. Another area of spatial statistics is the spatial point pattern, in which the location of points is studied. As locations of Visium spots are already known, spatial point pattern analyses are not that relevant to Visium, but they are relevant to highly multiplexed smFISH based spatial transcriptomics datasets.

# Compare with Space Ranger results
In the kallisto bustools paper, we compared the gene count matrices from Cell Ranger and kallisto bustools, and some simple comparisons are done in this tutorial as well.

In [ ]:
their_sce <- read10xCounts("their/D14.h5")

In [ ]:
their <- as(counts(their_sce), "dgCMatrix")
colnames(their) <- colData(their_sce)$Barcode
rownames(their) <- rowData(their_sce)$ID
their <- their[rowSums(their) > 0,]

In [ ]:
dim(D14_filtered)

In [ ]:
dim(their)

In [ ]:
prop_non0(D14_filtered)

In [ ]:
prop_non0(their)

So theirs is a bit sparser. How many genes are detected with both pipelines?

In [ ]:
colnames(their) <- str_remove(colnames(their), "-1$")

In [ ]:
bcs_overlap <- intersect(colnames(D14_filtered), colnames(their))
genes_overlap <- intersect(rownames(D14_filtered), rownames(their))

In [ ]:
upset(fromList(list(kallisto = rownames(D14_filtered),
                    spaceranger = rownames(their))))

Sounds good. The vast majority of genes do overlap. How well do barcodes in Space Ranger and from kallisto bustools correlate?

In [ ]:
# Using overlapping spots and genes
kb_overlap <- D14_filtered[genes_overlap, bcs_overlap]
sr_overlap <- their[genes_overlap, bcs_overlap]

As the matrices are pretty sparse, perhaps the correlation should be calculated only for genes detected by either pipeline so the 0's don't inflate the correlation coefficient.

In [ ]:
corrs <- map_dbl(bcs_overlap, ~ {
  kb_use <- kb_overlap[,.]
  sr_use <- sr_overlap[,.]
  inds <- kb_use > 0 | sr_use > 0
  cor(kb_use[inds], sr_use[inds])
})

In [ ]:
options(repr.plot.width=9, repr.plot.height=6)
hist(corrs, breaks = 50, cex.lab = 1.7, cex.axis = 1.5, cex.main = 1.7)

For almost all spots, Pearson's correlation with genes detected by either pipeline for each barcode is well over 0.95. How about using all overlapping genes?

In [ ]:
corrs2 <- map_dbl(bcs_overlap, ~ cor(kb_overlap[,.], sr_overlap[,.]))

In [ ]:
hist(corrs2, breaks = 50, cex.lab = 1.7, cex.axis = 1.5, cex.main = 1.7)

Good, the Pearson correlation is generally very high. Even the worst case isn't that bad. Does the correlation depend on locations?

In [ ]:
# The one with genes detected by either for each barcode
colData(spe2)$sr_corr <- corrs
# For plotting, remove outlier for better dynamic range
colData(spe2)$sr_corr_plt <- ifelse(corrs < 0.9, NA, corrs)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
vis_gene(spe2, "D14", "sr_corr_plt", assayname = "counts")

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
colData(spe2) %>% 
  as.data.frame() %>% 
  ggplot(aes(nCounts, sr_corr_plt)) +
  geom_point(alpha = 0.3) +
  labs(x = "Total counts per spot", y = "Pearson correlation")

While the correlation is generally very high, it is somewhat related to total counts, and the correlation tends to be higher in the atria and valves than in the ventricles here. I wonder if this is related to how kallisto bustools detect fewer transcripts from ribosomal protein genes and mouse olfactory genes as described in the [kallisto bustools paper](https://www.nature.com/articles/s41587-021-00870-2). 

# Data normalization

Data normalization is commonly performed on scRNA-seq datasets, to correct for technical effects on total counts per barcode on the one hand, and to stabilize variance and make the data look more normally distributed for downstream analysis methods that assume normal distribution or homoscedasticity on the other hand. 

I'm not sure about the biological relevance of total counts and the impact of technical differences between Visium and scRNA-seq. However, Visium data is usually normalized as in scRNA-seq in the literature. So to normalize or not to normalize? We would still normalize data here because the input of `BayesSpace` is PCA from log normalized data. Here we use `sctransform` to perform normalization; you actually don't have to use Seurat to use `sctransform`.

In [ ]:
sct <- vst(D14_filtered, n_genes = 3000, return_corrected_umi = TRUE)

SCTransform only kept genes detected in at least 5 cells.

In [ ]:
spe2 <- spe2[rownames(sct$y),]
logcounts(spe2) <- log1p(sct$umi_corrected)

In [ ]:
# Seurat style SCTransform highly variable genes
hvg <- rownames(sct$gene_attr)[order(sct$gene_attr[,"residual_variance"], decreasing = TRUE)]
hvg <- hvg[1:3000]

# Dimension Reduction and clustering
## Non-spatial
This part is the same as for scRNA-seq.

In [ ]:
reducedDim(spe2, "PCA") <- calculatePCA(sct$y[hvg,], ncomponents = 40, ntop = 3000,
                                        scale = TRUE)

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
plotPCA(spe2, colour_by = "nCounts") +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

With the `sctransform` normalization, total counts don't seem to be greatly contributing to the first two PCs.

In [ ]:
percent.var <- attr(reducedDim(spe2), "percentVar")
plot(percent.var, xlab="PC", ylab="Variance explained (%)",
     cex.lab = 1.7, cex.axis = 1.5)

The "elbow" is around PC 6, but to retain more info, we'll use 15 PCs.

In [ ]:
reducedDim(spe2, "PCA.elbow") <- reducedDim(spe2, "PCA")[,1:15]

In [ ]:
g <- buildSNNGraph(spe2, k = 10, use.dimred = "PCA.elbow")
set.seed(89)
colData(spe2)$louvain <- factor(cluster_louvain(g)$membership)

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
dittoDimPlot(spe2, "louvain", reduction.use = "PCA", size = 1.5) +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

In [ ]:
spe2 <- runUMAP(spe2, dimred = "PCA.elbow")

In [ ]:
dittoDimPlot(spe2, "louvain", reduction.use = "UMAP", size = 1.5) +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
vis_clus(spe2, "D14", "louvain", colors = dittoColors(1))

Here PCA, UMAP, and Louvain clustering did not use spatial information. Only the transcriptomes are used. While the Louvain clusters may delineate spatial regions in the tissue, such as the atria, the valves, and the right ventricle, sometimes spots of the same cluster are more spread out through the tissue and do not delineate a spatial region. Spatial regions can mean different cell types, in which case the transcriptome encodes spatial regions in some ways. PC1, which explains the most variance among the PCs, separates the valves from the other cells, and PC2 separates the atria from the other cells.

## Spatial
There are spatially informed dimension reduction methods, such as [geographically weighted PCA (GWPCA)](https://github.com/harvid50/GWPCA_Workshops/blob/master/GWPCA_IJGIS_preprint.pdf), in which PCA is performed at each location with its neighbors as defined by spatial weights. Each location would have a separate set of PCs, which can inform us of which variables are contributing the most to local variances around each location. Visualization is challenging even with only a few variables, and is much more so for spatial transcriptomics. There are other forms of spatially informed PCA, such as [sPCA](https://github.com/thibautjombart/adegenet/raw/master/tutorials/tutorial-spca.pdf), which is a fusion of PCA and Moran's I and returns large positive eigenvalues that indicate positive spatial autocorrelation and large negative eigenvalues that indicate negative spatial autocorrelation. See [this paper](https://mural.maynoothuniversity.ie/5850/1/CB_Analysis%20on%20Spatial%20Data.pdf) for more info on different types of spatial PCA applied to fields like geography and ecology, but not yet to spatial transcriptomics.

There are also a number of methods for spatially informed clustering for spatial transcriptomics, i.e. using transcriptomes to delineate anatomical regions in the tissue that might not be apparent from H&E tissue morphology. These methods often involve a Markov random field that favors the same cluster label for neighboring spots, so cluster assignment is spatially contiguous. `BayesSpace` is one of these methods, which we use in this tutorial because its principles are reasonable and it's easy to use and easy to install from Bioconductor. As there is yet a 3rd party comprehensive benchmark for spatial clustering methods in spatial transcriptomics, using `BayesSpace` here does not mean endorsing it as best practive. Here's the reference for `BayesSpace`:

> Zhao, E., Stone, M.R., Ren, X. et al. Spatial transcriptomics at subspot resolution with BayesSpace. Nat Biotechnol (2021). https://doi.org/10.1038/s41587-021-00935-2

Such spatial neighborhoods are not entirely the same as non-spatial clustering based on the transcriptome; they are different aspects of the tissue. Here's a geographical analogy: While it's fair to characterize Westwood, especially the part north of Wilshire Blvd, as the UCLA college town, and indeed many UCLA students live there and students do contribute to the ethos there, not everyone who lives in Westwood is a UCLA student, nor does every UCLA student live in Westwood. Non-students, such as restaurant staff and bus drivers, contribute to the unique ethos of the college town as well. The UCLA college town is a spatial region, and the UCLA students, as a demographic group, is analogous to a cell type.

Now let's use `BayesSpace` to identify spatial regions in this chicken heart. `BayesSpace` requires users to pre-define the number of clusters, and `qTune()` can search a range of number of clusters and help us to select a number that minimizes the negative log likelihood (i.e. maximize the log likelihood) of the model underlying `BayesSpace`. From Louvain clustering, 9 clusters seems reasonable, so we try a range around 9. The argument `d` is the number of top PCs to use. `BayesSpace` runs Markov chain Monte Carlo (MCMC) for fit its model, and in `qTune()`, the chain is run for fewer number of steps to get a tentative sense of different number of clusters before running the chain for a greater number of iterations to actually fit the model.

In [ ]:
set.seed(129)
system.time(spe2 <- qTune(spe2, qs = 7:15, platform = "Visium", d = 15, 
                          burn.in = 500, nrep = 2000))

In [ ]:
options(repr.plot.width=9, repr.plot.height=6)
qPlot(spe2) +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

This plot seems to call for a higher number of clusters. `BayesSpace` runs MCMC to estimate its parameters, so it takes a while to fit the model.

In [ ]:
set.seed(129)
system.time(spe2 <- spatialCluster(spe2, q = 14, d = 15, platform = "Visium"))

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
vis_clus(spe2, "D14", "spatial.cluster", colors = dittoColors(1))

`BayesSpace` clusters do indeed look more spatially contiguous. Some clusters have been eliminated. The clusters are the lower left ventricle, the upper left ventricle, the right ventricle, lining of the ventricles, the valves, the area surrounding the valves, the atria, and the septum at the atria. I wonder how this may or may not improve pathologist annotations based on H&E morphology.

Just like in the geographic world, where the boundaries of neighborhoods are often subject to dispute due to complexity of the real world, and just like how Louvain clustering of scRNA-seq data does not necessarily delineate new cell types, spatial regions identified by the spatial clustering packages are not definite. 

Using spatially informed clustering to identify spatial regions based on features is not new, nor is it unique to spatial transcriptomics. Such clustering has long been performed in geography. Examples of R packages for spatial clustering but not designed for spatial transcriptomics are [`ClustGeo`](https://arxiv.org/abs/1707.03897), which uses dissimilarity in both feature space and physical space for hierarchical clustering, and [`geocmeans`](https://cran.r-project.org/web/packages/geocmeans/vignettes/introduction.html), which performs spatially weighted c-means fuzzy clustering and only probabilistically assigns locations to clusters. Again, the `spatialGE` R package used the spatial hierarchical clustering method to identify spatial regions.

For spatial transcriptomics, methods that identify spatial regions typically only identify such regions at one spatial scale. However, multi-scale regions can be helpful. A geographic analogy is that Southern California (So-Cal) is a coarser scale region of California, and indeed Los Angeles (LA), Orange County, and San Diego have much in common in terms of climate and culture that set them apart from the Bay Area. However, LA itself can be partitioned into finer regions based on local ethos, climate, and socioeconomic class, such as Downtown LA, Koreatown, Eastside, Westside, and etc., each of which can be further partitioned, e.g. Westwood is part of Westside. Then merely finding So-Cal definitely misses a lot of information. Which scale of partition to find depends on the questions asked. An analyst of nation scaled data might only need So-Cal or LA county, while a tourist visiting LA would need the finer partitions. Another question here is whether merely subsetting the `SpatialExperiment` object for finer scale clustering is indeed sufficient, because regions adjacent to the region for which a more refined analysis is performed can still be relevant, because both in the tissue and in the city, spatially adjacent regions do influence each other. Furthermore, there may be more than one way to characterize the tissue by spatial regions, while the spatial region methods only assign one label to each spot.

How does the spatial clustering look in UMAP space?

In [ ]:
spe2$spatial.cluster <- factor(spe2$spatial.cluster)

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
dittoDimPlot(spe2, "spatial.cluster", reduction.use = "UMAP", size = 1.5) +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

It seems like a sensible way to visually cluster the spots in UMAP space, and alternative but sensible way compared to Louvain clustering shown earlier. How do the Louvain clusters and the `BayesSpace` clusters overlap?

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)
colData(spe2) %>% as.data.frame() %>%
  gather_set_data(x = c("louvain", "spatial.cluster"), id = "barcode") %>% 
  ggplot(aes(x, id = barcode, split = y, value = 1)) +
  geom_parallel_sets(aes(fill = spatial.cluster), alpha = 0.5) +
  geom_parallel_sets_axes(axis.width = 0.1, fill = "white", color = "gray") +
  geom_parallel_sets_labels(angle = 0, size = 5) +
  scale_x_discrete(labels = c("Louvain", "BayesSpace")) +
  scale_fill_manual(values = dittoColors(1), name = "BayesSpace") +
  theme_minimal() +
  theme(axis.text.y = element_blank(), panel.grid = element_blank(),
        axis.title = element_blank(), axis.text.x = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

Louvain clusters don't map cleanly to `BayesSpace` clusters, except the cluster for atria.

# Differential expression

Usually, for scRNA-seq, after clustering, differential expression (DE) between each cluster and the rest of the cells or between different clusters is performed to identify cluster marker genes, with which the cell type of each cluster is identified. 

## Non-spatial

Visium data can be analyzed in the scRNA-seq way, doing DE with Louvain clusters without spatial information. Here the DE method used is the Wilcoxon rank sum test, which is the default in Seurat. The package that performs basic scRNA-seq DE for `SingleCellExperiment` is `scran`, which does DE differently from Seurat's default; by default `scran` runs DE for each pair of clusters rather than between each cluster and the rest of the cells as in Seurat. The `pval.type = "all"` combines p-values of genes from the pairwise tests so that only genes highly specific to one cluster are significant. The `direction = "up"` means that only genes more highly expressed in one cluster than another are tested.


In [ ]:
markers_louvain <- findMarkers(spe2, groups = spe2$louvain, pval.type = "all",
                               test="wilcox", direction="up")

The output is a list of S4 `DataFrame`s, and each of the `DataFrame` is for one cluster, and has p-values and false discovery rate (FDR) for each gene tested involving that cluster.

In [ ]:
markers_louvain[[2]]

In [ ]:
class(markers_louvain)

Plot the log normalized expressions of the top ranking "marker" genes for each Louvain clustering.

In [ ]:
# Plot top gene for each cluster
genes_plot <- unique(map_chr(as.list(markers_louvain), ~ rownames(.[1,])))

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
multi_dittoPlot(spe2, group.by = "louvain", vars = genes_plot, ncol = 3) +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12),
        strip.text = element_text(size = 14))

Some of the "marker" genes are not specific to the cluster of interest. Where are those marker genes expressed in the tissue?

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
vis_clus(spe2, "D14", "louvain", colors = dittoColors(1))

In [ ]:
multi_vis_gene <- function(spe, sample, genes, ncol = 3) {
  plts <- map(genes, function(x) vis_gene(spe, sample, geneid = x))
  wrap_plots(plts, ncol = ncol, byrow = TRUE)
}

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
multi_vis_gene(spe2, "D14", genes_plot, ncol = 4)

## Spatial

An area in spatial transcriptomics data analysis is identifying spatially variable genes (SVG), which means genes whose expressions depend on spatial locations. However, as already seen, since in a complex organ like the heart, different regions have different cell types, non-spatial Louvain clusters do somewhat spatial regions, some Louvain cluster marker genes are spatially variable, because the cell type that expresses such genes are not ubiquitously found in the heart section. Then many SVGs are really about spatial locations of cell types and cell types can be captured by non-spatial clustering anyway. Then I wonder how much more information identifying SVGs with tools designed for this purpose adds to non-spatial DE. This might depend on the tissue, and in some tissues, such as already seen in the heart, and in the brain cortex and the intestine villi, presence of certain cell types depends on spatial location because these cell types need to be where they are for the tissues for function properly. A more interesting question is whether a gene's expression depends on spatial locations even within a cell type. However, this leads to the question of what we really mean by "cell type".

### With spatial clusters

We have the spatial clusters from `BayesSpace`, and we can perform non-spatial DE between each cluster and the rest of the spots. As the clusters tend to be spatially contiguous, cluster marker genes will be SVGs. Do DE genes from `BayesSpace` clusters differ from those from Louvain clusters? If so, then what does it mean?

Again, the geographical analogy might help. A non-student living in Westwood would have a very good chance to interact with UCLA students, take walks in the UCLA campus, and get influenced by the ethos emergent from not only students but also things like restaurants that students like. So being in Westwood influences traits of people living in Westwood even if they are not UCLA students. Would something similar happen in the tissue? Perhaps; the `Giotto` package can perform DE depending on proximity to another cell type. However, the problem with cell type is that Visium does not have single cell resolution and spots are very likely a mixture of different cell types.

In [ ]:
markers_bs <- findMarkers(spe2, groups = spe2$spatial.cluster, pval.type = "all",
                          test="wilcox", direction="up")

In [ ]:
genes_plot2 <- unique(map_chr(as.list(markers_bs), ~ rownames(.[1,])))

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
multi_dittoPlot(spe2, group.by = "spatial.cluster", vars = genes_plot2, ncol = 3) +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12),
        strip.text = element_text(size = 14))

It seems that the top ranking marker genes of the spatial clusters do tend to be more highly expressed in the cluster of interest than in other clusters. Again, where are they expressed in tissue?

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
vis_clus(spe2, "D14", "spatial.cluster", colors = dittoColors(1))

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
multi_vis_gene(spe2, "D14", genes_plot2)

### SPARK
A problem with clustering is that sometimes the very notion of discrete cell types is problematic. There may be a continuum of cell states that would not yield to the discrete pigeon holes. Similarly, there may be a continuum of spatial regions, which is observed in geographical space as community ethos often changes gradually in space, which causes disputes in how boundaries between neighborhoods should be defined. DE methods that don't rely on clustering have been developed in both scRNA-seq and spatial transcriptomics. In spatial transcriptomics, Gaussian process regression, with generalized generalized versions for Poisson or negative binomial data, is a common principle underlying the SVG methods. A gene is deemed a SVG if the model involving the Gaussian process to model spatial autocorrelation fits the data for the gene better than a model with only the linear covariates without the Gaussian process. SPARK is one of these Gaussian process regression based methods. Here's the reference of SPARK:

> Sun, S., Zhu, J. & Zhou, X. Statistical analysis of spatial expression patterns for spatially resolved transcriptomic studies. Nat Methods 17, 193–200 (2020). https://doi.org/10.1038/s41592-019-0701-7


Since a gene that is uniformly expressed or not expressed across the tissue section can't be spatially variable and SPARK takes a while to run, only the 3000 highly variable genes (non-spatial, from sctransform) are used for SPARK. SPARK is not on Bioconductor or CRAN and is on GitHub only. It requires its own data container object, though it would be nicer to reuse the `SpatialExperiment` or `Seurat` objects. Again, SPARK is used here because it's representative of Gaussian process regression based spatially variable genes methods, and using it here does not mean endorsing it as best practice.


In [ ]:
locations_df <- setNames(as.data.frame(colData(spe2))[,c("imagecol", "imagerow")],
                         c("x", "y"))

In [ ]:
spark <- CreateSPARKObject(as.matrix(logcounts(spe2)[hvg,]), location = locations_df)

Fitting the model takes a while.

In [ ]:
system.time(spark <- spark.vc(spark, lib_size = colData(spe2)$nCounts, 
                              num_core = 2, verbose = TRUE))

Randomly choose some spatially variable genes to plot.

In [ ]:
set.seed(70)
genes_spark <- spark@res_mtest %>% 
  slice_min(adjusted_pvalue, n = 1) %>% 
  rownames() %>% sample(4)

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
multi_vis_gene(spe2, "D14", genes_spark, ncol = 2)

### Gene patterns

Now with the SVGs, are there "archetypal" gene expression patterns? Clustering of the genes, whether using spatial information or not, is commonly used to identify such patterns. Here we can use hierarchical clustering and then calculate the "average" of each cluster to get some sense of the "archetypes".

In [ ]:
genes_spark2 <- spark@res_mtest %>% 
  filter(adjusted_pvalue < 0.05) %>% 
  rownames()
d <- dist(sct$y[genes_spark2,])
patterns <- hclust(d)
plot(patterns)

In [ ]:
spark_patterns <- cutree(patterns, k = 5)
table(spark_patterns)

In [ ]:
for (i in c(1:5)) {
  colData(spe2)[[paste0("means", i)]] <- colMeans(logcounts(spe2)[names(which(spark_patterns == i)),])
}

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)
multi_vis_gene(spe2, "D14", paste0("means", 1:5), ncol = 3)

We have already seen these "archetypal" gene patterns in the Louvain and `BayesSpace` cluster DE genes and SVGs found by SPARK: 

* Everywhere except the valves and the right ventricle
* The atria
* The valves
* Stronger in the right ventricle
* The ventricles

# Cell type deconvolution
Visium does not have single cell resolution. Suppose a cell is about 10 $\mu$m across on average, then several cells can fit into one Visium spot, whose diameter is 55 $\mu$m. For this dataset, as we only have the low resolution H&E image that does not show the nuclei clearly, we don't know the actual number of cells per spot. Many computational methods have been developed to estimate the proportion of cell types -- i.e. deconvolve the cell types -- in each Visium spot, by combining the non-single cell spatial data with scRNA-seq. Some were originally developed for bulk RNA-seq data, while some were designed for array based spatial transcriptomics techniques that don't have single cell resolution such as ST, Visium, and slide-seq. Some papers (including the paper where this dataset comes from) used Seurat to transfer cell type labels from scRNA-seq to Visium and used the transfer score of each cell type as a proxy to cell type proportions in each spot. Some methods specifically designed for cell type deconvolution use non-negative matrix factorization (NMF) where the factors stand for cell types and use non-negative least squares to assign cell types to spots. Some methods explicitly model the observed gene counts at each spots with a negative binomial or Poisson model. The method we use here, RCTD, uses the Poisson model, whose mean parameter weights the scRNA-seq cell type in each spot. Some of the methods impose a sparsity constraint on cell type assignment, as it's unlikely to get too many cell types per spot due to the limited number of cells that can fit into the spot. Again, using RCTD here does not mean endorsing it as best practice. Here's the reference of RCTD:

> Cable, D.M., Murray, E., Zou, L.S. et al. Robust decomposition of cell type mixtures in spatial transcriptomics. Nat Biotechnol (2021). https://doi.org/10.1038/s41587-021-00830-w

Again, as this tutorial is about Visium and scRNA-seq fastq file processing and EDA are covered in other tutorials on this website, we will skip those and use the Cell Ranger filtered gene count matrix and the authors' cell type annotations here as the scRNA-seq reference. There are two scRNA-seq datasets at D14, one for the left ventricle (LV), and one for the right (RV). Modeling the raw counts, RCTD expects raw counts as inputs for both scRNA-seq and Visium data.

In [ ]:
scRNAseq_metadata <- read_csv("https://raw.githubusercontent.com/madhavmantri/chicken_heart/master/data/scRNAseq_metadata.csv")

In [ ]:
lvsce <- read10xCounts("their/D14_LV.h5")
rvsce <- read10xCounts("their/D14_RV.h5")

In [ ]:
lv <- counts(lvsce)
rv <- counts(rvsce)
colnames(lv) <- lvsce$barcode
colnames(rv) <- rvsce$barcode
rownames(lv) <- lvsce$gene_search
rownames(rv) <- rvsce$gene_search
rm(lvsce); rm(rvsce)

In [ ]:
colnames(lv) <- paste0("D14-LV_", str_remove(colnames(lv), "-1$"))
colnames(rv) <- paste0("D14-RV_", str_remove(colnames(rv), "-1$"))

As seen from the gene patterns, left and right ventricles of the same heart do have differences in gene expression. Furthermore, here the LV and RV scRNA-seq datasets are in separate matrices, and as RCTD expects raw counts and data integration methods often don't return "batch corrected" raw counts, whatever potential batch effect between the two datasets is not corrected for. Therefore, "LV" and "RV" are added to the cell type annotations to mark the ventricles and to add the ventricle information to RCTD.

In [ ]:
ref_mat <- cbind(lv, rv)[genes_overlap,]
cell_type_df <- scRNAseq_metadata %>% 
  filter(day == "D14") %>% 
  mutate(celltype = case_when(
    str_detect(orig.ident, "LV$") ~ paste0(celltypes.0.5, "-LV"),
    str_detect(orig.ident, "RV$") ~ paste0(celltypes.0.5, "-RV")))

RCTD expects each cell type to have at least 25 cells in the scRNA-seq dataset, so cells in the D14 scRNA-seq datasets of cell types too rare in these datasets are removed.

In [ ]:
(n_cells <- table(cell_type_df$celltype))
celltypes_rm <- names(n_cells)[n_cells < 25]

In [ ]:
cell_type_df <- cell_type_df %>% 
  filter(!celltype %in% celltypes_rm)
cell_types_use <- as.factor(cell_type_df$celltype)
names(cell_types_use) <- cell_type_df$X1
ref_mat <- ref_mat[,cell_type_df$X1]
ncounts <- colSums(ref_mat)

Again, RCTD is not on Bioconductor or GitHub, and it uses its own data structure. It would be nice to reuse the `SpatialExperiment` or `Seurat` objects.

In [ ]:
ref <- Reference(ref_mat, cell_types_use, ncounts)

In [ ]:
puck <- SpatialRNA(locations_df, D14_filtered[genes_overlap,],
                   colSums(D14_filtered[genes_overlap,]))

In [ ]:
rctd <- create.RCTD(puck, ref, max_cores = 2)

With the RCTD object, we're ready to fit the model. Here `doublet_mode` is a way to add a sparsity constraint so each spot does not get implausibly many cell types. The RCTD paper used a slide-seq dataset. As each slide-seq beads haave diameter 10 $\mu$m, each bead most likely captures transcripts from 1 or 2 cells. So in "doublet" mode, each spot can have at most 2 cell types. "Doublet" mode also makes it possible to get decomposed counts for each of the cell types. However, as Visium spots are larger and most likely have more than 2 cells per spot, which means probably more than 2 cell types, "doublet" mode is inappropriate. There's also "multi" mode, which uses a greedy algorithm to find the number of cell types per spot (typically 3 or 4), though sometimes plausible cell types are excluded from the assignment. Finally, there's "full" mode, which does not constrain the number of cell types. 

In [ ]:
rctd <- run.RCTD(rctd, doublet_mode = "full")

Normalize the cell type scores so they are cell type proportions per spot and add up to 1.

In [ ]:
rctd_res <- rctd@results
norm_weights <- sweep(rctd_res$weights, 1, rowSums(rctd_res$weights), '/') 

Now plot the cell type proportions per spots. This can done by plotting little pie charts at each spot, and takes some data wrangling.

In [ ]:
cell_type_plt <- rctd@spatialRNA@coords
cell_type_plt <- cbind(cell_type_plt, as.data.frame(as.matrix(norm_weights)))

In [ ]:
cell_type_plt$barcode <- rownames(cell_type_plt)
cell_type_plt <- cell_type_plt %>% 
  pivot_longer(`Cardiomyocytes-1-LV`:`Vascular endothelial cells-RV`,
               names_to = "celltype", values_to = "proportion")

Also, make a palette that have similar colors for the same cell types for LV and RV. The RV ones are the pastel versions of the LV ones, except that one cell type is only abundant enough in the RV scRNA-seq dataset.

In [ ]:
lv_celltypes <- cell_type_names[str_detect(cell_type_names, "LV")]
rv_celltypes <- cell_type_names[str_detect(cell_type_names, "RV")]
pal_use <- c(setNames(brewer_pal(palette = "Set2")(8), lv_celltypes),
             setNames(brewer_pal(palette = "Pastel2")(8), rv_celltypes[-6]),
             setNames(brewer_pal(palette = "Accent")(6)[3], rv_celltypes[6]))
pal_use <- pal_use[sort(names(pal_use))]

The function `geom_arbc_bar()` from the `ggforce` package is used to plot those little pie charts. It takes a while to plot 1620 little pie charts, but it's not too bad.

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)
ggplot(cell_type_plt) +
  geom_arc_bar(aes(x0 = x, y0 = -y, r0 = 0, amount = proportion, fill = celltype,
                   r = 120), stat = "pie", size = 0) +
  scale_fill_manual(values = pal_use, name = "Cell type") +
  coord_equal() +
  theme_no_axes() +
  theme(panel.border = element_blank(),
        axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 14),
        legend.text = element_text(size = 12))

The little pie charts kind of show how cell types colocalize. Here LV cell types are often assigned to RV spots and vice versa, suggesting similarity between the LV and RV versions of each cell type and the lack of strong batch effect in the LV and RV scRNA-seq data. However, RV spots do tend to get a higher proportion of RV cell types. We also see that the valves have very disctinct cell type composition, where fibroblasts and TMSB4X high cells colocalize. The original paper of this dataset used cord plot to show cell type colocalization at the spots, which some of my colleagues find hard to read. Here we simply use a heatmap of the correlation matrix to show cell type colocalization.

In [ ]:
corr <- cor(as.matrix(norm_weights))
# Suppress the diagonal for better dynamical range in the heatmap
diag(corr) <- 0

Beause it's difficult to create a color bar legend with base R graphics, we use `ggplot2` to plot the heatmap, with the clustered reordering of rows and columns.

In [ ]:
# For row and column ordering
dendro <- as.dendrogram(hclust(dist(corr)))
ord <- order.dendrogram(dendro)

In [ ]:
as.data.frame(corr) %>% 
  mutate(celltype1 = rownames(.)) %>% 
  pivot_longer(-celltype1, names_to = "celltype2", values_to = "Pearson") %>% 
  mutate(celltype1 = fct_relevel(celltype1, sort(rownames(corr))[ord]),
         celltype2 = fct_relevel(celltype2, sort(rownames(corr))[ord])) %>% 
  ggplot(aes(celltype1, celltype2, fill = Pearson)) +
  geom_raster() +
  # Make the yellow the center of the palette
  scale_fill_distiller(limits = c(-max(abs(corr)), max(abs(corr))),
                       palette = "RdYlBu", direction = -1) +
  coord_equal() +
  theme(axis.text.x.bottom = element_text(angle = 90, hjust = 1, vjust = 0.5),
        axis.title = element_blank())

# Future directions

Many other analyses can be performed on Visium datasets. For instance, ligand-receptor pair databases and data analysis methods that utilize such databases can infer interactions between cell types. However, this is difficult for this dataset because the best ligand-receptor pair databases are for human and mice. While it's possible to build a ligan-receptor pair database for chicken, it's too much for this tutorial. Furthermore, with datasets from different stages of development, a spatiotemporal analysis can be performed. However, traditional methods from spatiotemporal statistics can't be easily applied here because it's already a challenge to match tissue regions across developmental stages. However, with kallisto bustools, RNA velocity analyses can be performed across developmental stages, in additional to trajectory inference.